In [1]:
from tensorflow.keras import models, layers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

In [16]:
df = pd.read_csv('data/titanic.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [18]:
del df['PassengerId']
del df['Name']
del df['Ticket']
del df['Cabin']

In [19]:
df.isnull().sum().sort_values(ascending=False).head(20)

Age         177
Embarked      2
Survived      0
Pclass        0
Sex           0
SibSp         0
Parch         0
Fare          0
dtype: int64

In [20]:
df = pd.get_dummies(df, dtype=int)
df = df.fillna(df.mean())

In [21]:
df_corr = df.corr()
df_corr.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
Survived,1.000000,-0.338481,-0.069809,-0.035322,0.081629,0.257307,0.543351,-0.543351,0.168240,0.003650,-0.155660
Pclass,-0.338481,1.000000,-0.331339,0.083081,0.018443,-0.549500,-0.131900,0.131900,-0.243292,0.221009,0.081720
Age,-0.069809,-0.331339,1.000000,-0.232625,-0.179191,0.091566,-0.084153,0.084153,0.032024,-0.013855,-0.027121
SibSp,-0.035322,0.083081,-0.232625,1.000000,0.414838,0.159651,0.114631,-0.114631,-0.059528,-0.026354,0.070941
Parch,0.081629,0.018443,-0.179191,0.414838,1.000000,0.216225,0.245489,-0.245489,-0.011069,-0.081228,0.063036


In [22]:
df_corr_sort = df_corr.sort_values('Survived', ascending=False)
df_corr_sort['Survived'].head(10)

Survived      1.000000
Sex_female    0.543351
Fare          0.257307
Embarked_C    0.168240
Parch         0.081629
Embarked_Q    0.003650
SibSp        -0.035322
Age          -0.069809
Embarked_S   -0.155660
Pclass       -0.338481
Name: Survived, dtype: float64

In [23]:
cols_train = ['Sex_female', 'Fare', 'Embarked_C', 'Parch']
X_train_pre = df[cols_train]
y = df['Survived'].values

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X_train_pre, y, test_size=0.2)

In [25]:
X_train.shape

(712, 4)

In [28]:
model = models.Sequential()
model.add(layers.Dense(30, input_dim=4, activation='relu'))
model.add(layers.Dense(12, activation='relu'))
model.add(layers.Dense(8, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 30)                150       
                                                                 
 dense_13 (Dense)            (None, 12)                372       
                                                                 
 dense_14 (Dense)            (None, 8)                 104       
                                                                 
 dense_15 (Dense)            (None, 1)                 9         
                                                                 
Total params: 635
Trainable params: 635
Non-trainable params: 0
_________________________________________________________________


In [29]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=20)

In [30]:
history = model.fit(X_train, y_train, epochs=1000, batch_size=500, validation_split=0.25, callbacks=[early_stopping])

Epoch 1/1000
2/2 [==============================] - 1s 222ms/step - loss: 1.0725 - accuracy: 0.3858 - val_loss: 1.0381 - val_accuracy: 0.4157
Epoch 2/1000
2/2 [==============================] - 0s 30ms/step - loss: 0.9433 - accuracy: 0.3858 - val_loss: 0.8881 - val_accuracy: 0.4157
Epoch 3/1000
2/2 [==============================] - 0s 28ms/step - loss: 0.8170 - accuracy: 0.3858 - val_loss: 0.7601 - val_accuracy: 0.4157
Epoch 4/1000
2/2 [==============================] - 0s 28ms/step - loss: 0.7128 - accuracy: 0.3876 - val_loss: 0.6789 - val_accuracy: 0.7079
Epoch 5/1000
2/2 [==============================] - 0s 28ms/step - loss: 0.6795 - accuracy: 0.7172 - val_loss: 0.6705 - val_accuracy: 0.6966
Epoch 6/1000
2/2 [==============================] - 0s 28ms/step - loss: 0.7197 - accuracy: 0.6517 - val_loss: 0.6717 - val_accuracy: 0.6685
Epoch 7/1000
2/2 [==============================] - 0s 28ms/step - loss: 0.7281 - accuracy: 0.6517 - val_loss: 0.6672 - val_accuracy: 0.6966
Epoch 8/1000

Epoch 59/1000
2/2 [==============================] - 0s 29ms/step - loss: 0.5963 - accuracy: 0.7584 - val_loss: 0.6453 - val_accuracy: 0.7135
Epoch 60/1000
2/2 [==============================] - 0s 28ms/step - loss: 0.5928 - accuracy: 0.7528 - val_loss: 0.6391 - val_accuracy: 0.7022
Epoch 61/1000
2/2 [==============================] - 0s 28ms/step - loss: 0.5890 - accuracy: 0.7566 - val_loss: 0.6324 - val_accuracy: 0.7135
Epoch 62/1000
2/2 [==============================] - 0s 27ms/step - loss: 0.5855 - accuracy: 0.7715 - val_loss: 0.6272 - val_accuracy: 0.7022
Epoch 63/1000
2/2 [==============================] - 0s 28ms/step - loss: 0.5845 - accuracy: 0.7547 - val_loss: 0.6247 - val_accuracy: 0.6910
Epoch 64/1000
2/2 [==============================] - 0s 28ms/step - loss: 0.5840 - accuracy: 0.7622 - val_loss: 0.6241 - val_accuracy: 0.6910
Epoch 65/1000
2/2 [==============================] - 0s 27ms/step - loss: 0.5828 - accuracy: 0.7622 - val_loss: 0.6248 - val_accuracy: 0.6910
Epoch 

2/2 [==============================] - 0s 27ms/step - loss: 0.5351 - accuracy: 0.7715 - val_loss: 0.6229 - val_accuracy: 0.6910
Epoch 117/1000
2/2 [==============================] - 0s 29ms/step - loss: 0.5361 - accuracy: 0.7790 - val_loss: 0.6179 - val_accuracy: 0.6910
Epoch 118/1000
2/2 [==============================] - 0s 29ms/step - loss: 0.5326 - accuracy: 0.7734 - val_loss: 0.6086 - val_accuracy: 0.6966
Epoch 119/1000
2/2 [==============================] - 0s 28ms/step - loss: 0.5281 - accuracy: 0.7846 - val_loss: 0.6034 - val_accuracy: 0.6910
Epoch 120/1000
2/2 [==============================] - 0s 29ms/step - loss: 0.5258 - accuracy: 0.7978 - val_loss: 0.6049 - val_accuracy: 0.6966
Epoch 121/1000
2/2 [==============================] - 0s 29ms/step - loss: 0.5255 - accuracy: 0.7884 - val_loss: 0.6066 - val_accuracy: 0.7022
Epoch 122/1000
2/2 [==============================] - 0s 29ms/step - loss: 0.5250 - accuracy: 0.7846 - val_loss: 0.6046 - val_accuracy: 0.6966
Epoch 123/1000

In [15]:
score = model.evaluate(X_test, y_test)
print('Test Accuracy:', score[1])

6/6 [==============================] - 0s 1ms/step - loss: 8.9649 - accuracy: 0.4022
Test Accuracy: 0.40223464369773865
